In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os
import optuna

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os
import optuna

In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v13.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v13.csv")



In [ ]:
# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]
# df = challenge_set_updated.sample(frac=0.001)

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']



In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Handle categorical columns by Label Encoding
label_encoder = LabelEncoder()

# Specify the columns that are categorical
categorical_cols = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_season', 
                    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction', 
                    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight']

# Convert these categorical columns into numerical form using Label Encoding
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Now, you can proceed with training LightGBM with this processed data


In [ ]:

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor


# Define the objective function
def objective(trial):
    # Sample hyperparameters for LightGBM
    lgb_params = {
        'n_estimators': trial.suggest_int('lgb_n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('lgb_learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('lgb_max_depth', 3, 10),
        'num_leaves': trial.suggest_int('lgb_num_leaves', 31, 256),
    }

    # Sample hyperparameters for XGBoost
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10),
        'subsample': trial.suggest_float('xgb_subsample', 0.5, 1.0),
    }

    # Sample hyperparameters for CatBoost
    cat_params = {
        'n_estimators': trial.suggest_int('cat_n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('cat_learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('cat_depth', 3, 10),
    }

    # Initialize the models with sampled hyperparameters
    lgb_model = LGBMRegressor(**lgb_params)
    xgb_model = XGBRegressor(**xgb_params, objective='reg:squarederror')
    cat_model = CatBoostRegressor(**cat_params, verbose=0)

    # Ensemble the models using VotingRegressor
    ensemble_model = VotingRegressor(
        estimators=[
            ('lgb', lgb_model),
            ('xgb', xgb_model),
            ('cat', cat_model)
        ]
    )

    # Split the dataset (use your own dataset instead of X, y)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the ensemble model
    ensemble_model.fit(X_train, y_train)

    # Make predictions
    y_pred = ensemble_model.predict(X_valid)

    # Calculate RMSE (you can use any other metric like R^2, MAE, etc.)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    
    return rmse

In [ ]:
# Create an Optuna study object and optimize
study = optuna.create_study(direction='minimize')  # 'minimize' for minimizing RMSE
study.optimize(objective, n_trials=100)

# Print the best trial and its hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value (RMSE): {trial.value}")
print(f"  Params: {trial.params}")


In [ ]:
# Use the best hyperparameters to create models
best_params = study.best_trial.params

# Initialize models with the best parameters
lgb_model = LGBMRegressor(
    n_estimators=best_params['lgb_n_estimators'],
    learning_rate=best_params['lgb_learning_rate'],
    max_depth=best_params['lgb_max_depth'],
    num_leaves=best_params['lgb_num_leaves']
)

xgb_model = XGBRegressor(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    subsample=best_params['xgb_subsample'],
    objective='reg:squarederror'
)

cat_model = CatBoostRegressor(
    n_estimators=best_params['cat_n_estimators'],
    learning_rate=best_params['cat_learning_rate'],
    depth=best_params['cat_depth'],
    verbose=0
)

# Ensemble the models
ensemble_model = VotingRegressor(
    estimators=[
        ('lgb', lgb_model),
        ('xgb', xgb_model),
        ('cat', cat_model)
    ]
)

# Train the ensemble model on the entire dataset
ensemble_model.fit(X, y)



In [ ]:

# Make predictions
y_pred = ensemble_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")


In [ ]:
import joblib

# Save the ensemble model to a file
model_filename = 'ensemble_model.pkl'
joblib.dump(ensemble_model, model_filename)

print(f"Model saved to {model_filename}")


In [ ]:
from sklearn.preprocessing import LabelEncoder

submission_set_features = submission_set_updated.iloc[:,:-1]

# Now you can use the model to make predictions
submission_set['tow'] = ensemble_model.predict(submission_set_features)
print(submission_set)


In [ ]:
import os
from datetime import datetime

# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Define a timestamp for the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")

# Assuming submission_set is a DataFrame, save it to CSV
submission_set.to_csv(submission_file, index=False)

print(f"Submission saved to {submission_file}")
